# BT2 - RANSAC y Transformada de Hough

## Autores

* **Juan Diego Gallego Nicolás (jdiego.gallego@um.es)**
* **Óscar Vera López (oscar.veral@um.es)**
  
## Fecha

**27 de Octubre de 2025**

***

## Índice

### 1. Introducción al notebook

### 2. Configuración e Inicialización

* 2.1. Importación de librerías
* 2.2. Presentación del dataset imágenes

### 3. Preprocesamiento y detección de bordes

### 4. Detección de rectas

* 4.1. Implementación y aplicación de **RANSAC** para rectas
* 4.2. Implementación y aplicación de **Transformada de Hough** para Rectas

### 5. Detección de polígonos

### 6. Detección de circunferencias

* 6.1. Implementación y aplicación de **RANSAC** para rectas
* 6.2. Implementación y aplicación de **Transformada de Hough** para Rectas

### 7. Comparaciones con OpenCV

### 8. Puntos aplicados y conclusiones

### Uso de IA

### Bibliografía

***

# 1. Introducción al notebook

Este notebook es la segunda parte de una serie de cinco que constituye nuestra práctica para la asignatura de Visión Artificial del Máster en Inteligencia Artificial de la Universidad de Murcia. Alguno de los métodos/resultados que aquí se utilizan pueden estar comentados en un notebook anterior. Para el correcto funcionamiento del mismo es necesario instalar el paquete del proyecto siguiendo las instrucciones del [repositorio de GitHub]{https://github.com/oscarveral/vision.git}.

Aquí tratamos la detección de rectas y de otras formas geométricas características en el ámbito de las señales de tráfico, candidatas idóneas sobre el papel para aplicar las técnicas vistas en la asignatura. Comenzamos presentando el conjunto de imágenes que servirán como objeto para la aplicación de **RANSAC** (Random Sample Consensus) y la **HT** (Hough Transform). Seguidamente pasamos a la preparación y transformación de las imágenes, principalmente mediante la aplicación del **filtro de Canny** implementado en el notebook BT1 para la detección de bordes. Los siguientes puntos contienen la explicación y aplicación de nuestras implementaciones de RANSAC y HT aplicadas al dataset procesado. A contrinuación, comparamos nuestros resultados con los que obtiene otra libería estandarizada.

Al final del documento, incluimos un apartado de conclusiones y justificación de los ítems de bloques cubiertos por nuestro trabajo. También dejamos un párrafo explicando el papel de la IA generativa en la elaboración del mismo.